In [ ]:
import sys
sys.path.append('/opt/anaconda3/lib/python38.zip')
sys.path.append('/opt/anaconda3/lib/python3.8')
sys.path.append('/opt/anaconda3/lib/python3.8/lib-dynload')
sys.path.append('/home/oytj/.local/lib/python3.8/site-packages')
sys.path.append('/opt/anaconda3/lib/python3.8/site-packages')

import librosa
import librosa.display
from matplotlib import pyplot as plt 
import IPython
import math
import numpy as np
import os
import random
import time
import numpy as np
import cv2 as cv
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch import optim
from torch.autograd import Variable
import pyworld
import pysptk
from tqdm import tqdm
import model
from model import wav2spectrum
from model import spectrum2wav

In [ ]:
#读取模型
loadNetName='./trainedModel/Gan4-last20210205.pth'
savedNet=torch.load(loadNetName)
generator=model.Generator().cpu()
generator.load_state_dict(savedNet['netG'])

In [ ]:
#读取wav文件，并保存为npy
def wav2npy():
    noisyPath='%s/%s/%s/'%(L0Dir,L1Dir,noisyDir)
    labelPath='%s/%s/%s/'%(L0Dir,L1Dir,labelDir)
    stylePath='%s/%s/%s/'%(L0Dir,L1Dir,styleDir)
    noisyNpyPath='%s/%s/temp/%s_npy/'%(L0Dir,L1Dir,noisyDir)
    labelNpyPath='%s/%s/temp/%s_npy/'%(L0Dir,L1Dir,labelDir)
    styleNpyPath='%s/%s/temp/%s_npy/'%(L0Dir,L1Dir,styleDir)
    if not os.path.exists(noisyNpyPath):
        os.makedirs(noisyNpyPath) 
    if not os.path.exists(labelNpyPath):
        os.makedirs(labelNpyPath) 
    if not os.path.exists(styleNpyPath):
        os.makedirs(styleNpyPath) 
    filenames=os.listdir(noisyPath)
    filenames.sort()
    for filename in tqdm(filenames,desc='Loading '+noisyPath):
        src_voc, sr = librosa.load(noisyPath+filename,sr=None)#读取
        np.save(noisyNpyPath+filename+'.npy',src_voc)    
    filenames=os.listdir(labelPath)
    filenames.sort()
    for filename in tqdm(filenames,desc='Loading '+labelPath):
        src_voc, sr = librosa.load(labelPath+filename,sr=None)#读取
        np.save(labelNpyPath+filename+'.npy',src_voc)    
    filenames=os.listdir(stylePath)
    filenames.sort()
    for filename in tqdm(filenames,desc='Loading '+stylePath):
        src_voc, sr = librosa.load(stylePath+filename,sr=None)#读取
        np.save(styleNpyPath+filename+'.npy',src_voc)  
        

In [ ]:
#读取npy.产生输出
def ssu():
    noisyNpyPath='%s/%s/temp/%s_npy/'%(L0Dir,L1Dir,noisyDir)
    labelNpyPath='%s/%s/temp/%s_npy/'%(L0Dir,L1Dir,labelDir)
    styleNpyPath='%s/%s/temp/%s_npy/'%(L0Dir,L1Dir,styleDir)
    savePath='%s/%s/clean'%(L0Dir,L1Dir)
    if not os.path.exists(savePath):
        os.makedirs(savePath) 
    #saveFile = open(savePath,'w')
    filenames=os.listdir(noisyNpyPath)
    filenames.sort()
    for filename in tqdm(filenames,desc='Creating %s'%(savePath)):
        noisy=np.load(noisyNpyPath+filename)
        label=np.load(labelNpyPath+filename)
        style=np.load(styleNpyPath+filename)
        noisy=wav2spectrum(noisy)#转频谱
        noisy=torch.from_numpy(noisy.reshape(1,1,noisy.shape[0],noisy.shape[1]))#转tensor
        style=wav2spectrum(style)#转频谱
        style=torch.from_numpy(style.reshape(1,1,style.shape[0],style.shape[1]))#转tensor
        out=generator(noisy,style)
        clean=out.detach().numpy()[0,0]
        clean=spectrum2wav(clean)
        savename=os.path.splitext(filename)[0]
        librosa.output.write_wav(savePath+'/'+savename,clean,16384,norm=False)


In [ ]:
L0Dir='data/test'
L1Dir='THCHS30'
noisyDir='noisy'
labelDir='label'
styleDir='style'
wav2npy()
ssu()

In [ ]:
L0Dir='data/test'
L1Dir='ST-CMDS'
noisyDir='noisy'
labelDir='label'
styleDir='style'
wav2npy()
ssu()

In [ ]:
L0Dir='data/test'
L1Dir='libriTTS'
noisyDir='noisy'
labelDir='label'
styleDir='style'
wav2npy()
ssu()

In [ ]:
L0Dir='data/test'
L1Dir='VCTK-Corpus'
noisyDir='noisy'
labelDir='label'
styleDir='style'
wav2npy()
ssu()

In [ ]:
L0Dir='data/test'
L1Dir='VCTK_Demand'
noisyDir='noisy'
labelDir='label'
styleDir='style'
wav2npy()
ssu()

In [ ]:
L0Dir='data/test'
L1Dir='VCTK_NoiseX92'
noisyDir='noisy'
labelDir='label'
styleDir='style'
wav2npy()
ssu()

In [ ]:
L0Dir='data/test'
L1Dir='TIMIT'
noisyDir='noisy'
labelDir='label'
styleDir='style'
wav2npy()
ssu()